In [1]:
from omop_alchemy.cdm.registry import CDMModelRegistry
from omop_alchemy.cdm.specification import TABLE_LEVEL_CSV, FIELD_LEVEL_CSV
from omop_alchemy import configure_logging, get_engine_name, load_environment, TEST_PATH, ROOT_PATH
from omop_alchemy.cdm.base import bootstrap
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker

configure_logging()
load_environment()

engine_string = get_engine_name()
engine = sa.create_engine(engine_string, future=True, echo=False)
registry = CDMModelRegistry(cdm_version="5.4")

registry.load_specs(
    table_csv=TABLE_LEVEL_CSV,
    field_csv=FIELD_LEVEL_CSV,
)

registry.discover_models("omop_alchemy.model")
bootstrap(engine, create=True)

2026-01-02 11:31:09,629 | INFO     | omop_alchemy.omop_alchemy.config | Environment variables loaded from .env file
2026-01-02 11:31:09,630 | INFO     | omop_alchemy.omop_alchemy.config | Database engine configured
2026-01-02 11:31:09,676 | INFO     | omop_alchemy.omop_alchemy.cdm.base.declarative | Bootstrapping OMOP schema (create=True)
2026-01-02 11:31:09,676 | INFO     | omop_alchemy.omop_alchemy.cdm.base.declarative | Schema creation enabled


In [2]:
list(registry.known_tables())[:5]

['visit_detail', 'person', 'observation', 'vocabulary', 'payer_plan_period']

In [3]:
report = registry.validate(engine=engine, check_domain_semantics=True)

2026-01-02 11:31:15,202 | INFO     | omop_alchemy.omop_alchemy.cdm.registry.registry | Starting CDM model validation (types=True, fks=True, domain_semantics=True, engine=True)
2026-01-02 11:31:15,204 | INFO     | omop_alchemy.omop_alchemy.cdm.registry.registry | Validating database schema against ORM definitions
2026-01-02 11:31:15,210 | WARNING  | omop_alchemy.omop_alchemy.cdm.registry.registry | No engine provided; skipping domain existence check for DomainRule on person.gender_concept_id
2026-01-02 11:31:15,211 | WARNING  | omop_alchemy.omop_alchemy.cdm.registry.registry | No engine provided; skipping domain existence check for DomainRule on person.race_concept_id
2026-01-02 11:31:15,211 | WARNING  | omop_alchemy.omop_alchemy.cdm.registry.registry | No engine provided; skipping domain existence check for DomainRule on person.ethnicity_concept_id
2026-01-02 11:31:15,211 | WARNING  | omop_alchemy.omop_alchemy.cdm.registry.registry | No engine provided; skipping domain existence check 

In [ ]:
print(report.render_text())

In [ ]:
if not report.is_valid():
    raise ValueError("CDM model validation failed")

In [ ]:
for table, spec in registry._table_specs.items():
    print(f"{table}: {spec.is_required}")